# CTEs (WITH Clauses) com DuckDB

* [WITH Clause](https://duckdb.org/docs/stable/sql/query_syntax/with)

In [1]:
import duckdb

In [2]:
conn = duckdb.connect()

In [3]:
conn.execute("""
    CREATE OR REPLACE TABLE vendas (
        id INTEGER,
        vendedor TEXT,
        regiao TEXT,
        valor NUMERIC,
        data DATE
    );
""")

In [4]:
dados = [
    (1, 'Ana',    'Sul',   1000, '2024-01-01'),
    (2, 'Bruno',  'Sul',   1500, '2024-01-02'),
    (3, 'Carla',  'Norte', 1200, '2024-01-03'),
    (4, 'Ana',    'Sul',   900,  '2024-01-04'),
    (5, 'Bruno',  'Sul',   2000, '2024-01-05'),
    (6, 'Carla',  'Norte', 1100, '2024-01-06'),
    (7, 'Ana',    'Sul',   1300, '2024-01-07')
]

In [5]:
conn.executemany(
    "INSERT INTO vendas VALUES (?, ?, ?, ?, ?);",
    dados
)

# Maior venda por região usando ROW_NUMBER

In [7]:
display(conn.execute("""
    SELECT 
        *
    FROM (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY regiao ORDER BY valor DESC) AS posicao
        FROM vendas
    ) WHERE posicao = 1;
""").df())

,id,vendedor,regiao,valor,data,posicao
0,3,Carla,Norte,1200.0,2024-01-03,1
1,5,Bruno,Sul,2000.0,2024-01-05,1


In [8]:
display(conn.execute("""
    WITH vendas_ranked AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY regiao ORDER BY valor DESC) AS posicao
        FROM vendas
    )
    SELECT
        *
    FROM vendas_ranked
    WHERE posicao = 1;
""").df())

,id,vendedor,regiao,valor,data,posicao
0,3,Carla,Norte,1200.0,2024-01-03,1
1,5,Bruno,Sul,2000.0,2024-01-05,1


# Acima da média por região

In [9]:
display(conn.execute("""
    WITH media_por_regiao AS (
        SELECT
            regiao,
            AVG(valor) AS media_vendas
        FROM vendas
        GROUP BY regiao
    )
    SELECT
        v.vendedor,
        v.valor,
        v.regiao,
        m.media_vendas
    FROM vendas v
    JOIN media_por_regiao m ON v.regiao = m.regiao
    WHERE v.valor > m.media_vendas;
""").df())

,vendedor,valor,regiao,media_vendas
0,Bruno,1500.0,Sul,1340.0
1,Carla,1200.0,Norte,1150.0
2,Bruno,2000.0,Sul,1340.0


# CTEs encadeadas: média e soma por região

In [10]:
display(conn.execute("""
    WITH media AS (
        SELECT 
            regiao,
            AVG(valor) AS media_valor
        FROM vendas
        GROUP BY regiao
    ),
    soma AS (
        SELECT
            regiao,
            SUM(valor) AS soma_valor
        FROM vendas
        GROUP BY regiao
    )
    SELECT
        m.regiao,
        m.media_valor,
        s.soma_valor
    FROM media m
    JOIN soma s ON m.regiao = s.regiao;
""").df())

,regiao,media_valor,soma_valor
0,Norte,1150.0,2300.0
1,Sul,1340.0,6700.0


In [ ]:
conn.close()